# Optimizing models with tensorboard
code adopted from https://pythonprogramming.net/tensorboard-optimizing-models-deep-learning-python-tensorflow-keras/
modified by Precious Jatau

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import pickle
import time

# dense layers is number of fully connected (FC) layers
# conv layers is number of convolutional layers 
# layer sizes is number of nodes for FC and number of filters for conv
dense_layers = [0, 1, 2]
layer_sizes = [64] # [32, 64, 128]
conv_layers = [2] # [1, 2, 3]
num_epochs = 4

# load data
X = pickle.load(open("X.pickle","rb"))
Y = pickle.load(open("Y.pickle","rb"))

# normalize X using min-max scaling
X = X/255.0

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            tensorboard = TensorBoard(log_dir = 'logs\{}'.format(NAME))
            
            model = Sequential()
            
            # convolutional layers
            model.add(Conv2D(layer_size, (3,3), input_shape = X.shape[1:]))
            model.add(Activation("relu"))
            model.add(MaxPooling2D(pool_size = (2,2)))

            for l in range(conv_layer - 1):            
                model.add(Conv2D(layer_size, (3,3)))
                model.add(Activation("relu"))
                model.add(MaxPooling2D(pool_size = (2,2)))

            # fully connected layers
            model.add(Flatten())            
            for l in range(dense_layer - 1):
                model.add(Dense(layer_size))
                model.add(Activation("relu"))

            # output layer
            model.add(Dense(1))
            model.add(Activation("sigmoid"))

            # compile model
            model.compile(loss = "binary_crossentropy",
                         optimizer = "adam",
                         metrics = ['accuracy'])

            model.fit(X, Y, batch_size = 32, epochs = num_epochs, validation_split = 0.1, callbacks = [tensorboard])